In [6]:
import numpy as np
import matplotlib.pyplot as plt

from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS

from qiskit  import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit.opflow import AerPauliExpectation
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.connectors import TorchConnector

# Additional torch-related imports
from torch import cat, no_grad, manual_seed
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (Module, Conv2d, Linear, Dropout2d, NLLLoss,
                     MaxPool2d, Flatten, Sequential, ReLU)
import torch.nn.functional as F
from torch.autograd import Function


import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/Users/trongduong/Dropbox/URP project/Code/PQC_Composer')
from utility.quantum_nn import  QuantumNeuralNetwork
from utility.ansatz_template import AnsatzTemplate
from utility.data_encoding import FeatureMap

In [5]:
class HybridFunction(Function):
    """ Hybrid quantum - classical function definition """
    
    @staticmethod
    def forward(ctx, batch_data, param, model, observable):
        """ Forward pass computation """
        
        ctx.batch_data = batch_data
        ctx.model = model
        ctx.observable = observable
        
        num_inputs = batch_data.shape[0]
        grid_inputs = batch_data.toarray()
        grid_params = np.tile(param.toarray(),(num_inputs,1))
        
        if isinstance(ctx.observable, str):
            obs = ctx.observable
        else:
            obs = [ctx.observable]
        
                
        exp_results = ctx.model.forward(grid_inputs, grid_params, observables=[ctx.observable])

        result = torch.tensor(exp_results).to(device)

        ctx.save_for_backward(param, result)
        
        return result
    
    
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """

        param, result = ctx.saved_tensors
        param_array = param.toarray()
        batch_data_array = ctx.batch_data.toarray()
        
        if isinstance(ctx.observable, str):
            obs = ctx.observable
        else:
            obs = [ctx.observable]
        
        
        _, num_params = params_array.shape
        gradients = ctx.model.get_gradients(grid_inputs, grid_params, observables=[ctx.observable])
        
        
        gradients = torch.tensor(gradients.reshape(ctx.batch_data.shape[0], num_params)).to(device)
        
        return None, None, gradients.float() * grad_output.float(), None, None, None, None

NameError: name 'Function' is not defined

In [ ]:
class Hybrid(nn.Module):
    """ Hybrid quantum - classical layer definition """
    
    def __init__(self, n_qubits, k_qubits, layers, ansatz_id, test, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.autoencoder = AutoencoderQuantumCircuit(n_qubits, k_qubits, layers, ansatz_id, backend=backend, shots=shots)
        self.shift = shift
        self.ansatz_id = ansatz_id
        self.test = test
        
        self.num_qubits = n_qubits + k_qubits
        if ansatz_id == 6:
            self.num_params = (self.num_qubits ** 2 + 3*self.num_qubits) * layers
        elif ansatz_id == 9:
            self.num_params = self.num_qubits * layers
        else:
            raise Exception("Invalid circuit ID")
        
        self.params = nn.Parameter(data=torch.rand((1,self.num_params), dtype=torch.float), requires_grad=True)
        
    def forward(self, batch_data):
        return HybridFunction.apply(batch_data, self.num_qubits, self.params, self.ansatz_id, self.test, self.autoencoder, self.shift)b

In [ ]:
class Net(nn.Module):
    def __init__(self, n_qubits, k_qubits, layers, ansatz_id=None, test='Hadamard', backend=provider.get_backend("ibmq_qasm_simulator"), shots=100, shift=np.pi / 2):
        super(Net, self).__init__()
        self.hybrid = Hybrid(n_qubits,k_qubits,layers,
                             ansatz_id, test,
                             backend=backend,
                             shots=shots,
                             shift=shift)

    def forward(self, x):
        #print("1: ", x.shape)
        x = self.hybrid(x.detach())
        #print("2: ", x.shape)
        return x

In [ ]:
def FLoss(output, weight=None):
    if weight is None:
        weight = np.ones(len(output)) / len(output)
    else:
        assert type(weight) == np.ndarray
        assert weight.shape == (len(outputs),)
        assert sum(weight) == 1
        
    #print("FLoss output = {}".format(output))
    weight = torch.tensor(weight.reshape(len(output),1)).to(device)
    loss = 1 - torch.sum(output * weight)
    return loss